In [4]:
import os
import json
import argparse
import logging
import pickle
import random

import pandas as pd
import torch
from torch.utils import data
import lightning.pytorch as pl
from lightning.pytorch.loggers.csv_logs import CSVLogger

from proteinclip import data_utils, fasta_utils, swissprot, hparams
from proteinclip import contrastive

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.96s/it]
Device set to use cuda:0


In [3]:
os.chdir(os.path.join(os.getcwd(), '../'))

In [6]:
struct = pd.read_pickle('../structural_embeddings_0_2000.pkl')

In [43]:
args = {
    'structural_embed':'../structural_embeddings_0_2000.pkl',
    'proteinclip_embed':'protclip_embed_dataset.parquet',
    'unitnorm': True,
    'batch_size': 64,
    'dim': 128,
    'nhidden': 1,
    'learning_rate':1e-4,
    'out':'training',
    'name':'testrun',
    'max_epochs': 1000,
    
}

In [ ]:
pl.seed_everything(seed=6489)


In [10]:
structural_embeddings = None
with open(args['structural_embed'], "rb") as f:
    structural_embeddings = pickle.load(f)
structural_embeddings = dict(zip(
    structural_embeddings["protein_id"],
    structural_embeddings["embedding"]
))

In [15]:
proteinclip_embeddings = pd.read_parquet(args['proteinclip_embed'])
proteinclip_embeddings = dict(zip(
    proteinclip_embeddings["id"],
    proteinclip_embeddings["proteinclip_embed"]
))

In [16]:
shared_keys = sorted(set(proteinclip_embeddings.keys()).intersection(set(structural_embeddings.keys())))


In [19]:
random.seed(42)  # Set your desired seed here
random.shuffle(shared_keys)


In [25]:
dset = data_utils.CLIPDataset(
    pairs=shared_keys,
    map1=proteinclip_embeddings,
    map2=structural_embeddings,
    enforce_unit_norm=args['unitnorm'],
)

Checking for missing/zero embeddings: 100%|██████████| 1997/1997 [00:00<00:00, 23063.08it/s]
INFO:root:Trimmed 0 pairs with missing/zero embeddings, 1997 remain.


In [27]:
split_indices = data_utils.random_split(len(dset), [0.9, 0.05, 0.05])
dset_splits = [data.Subset(dset, idx) for idx in split_indices]

In [31]:
# Create data loaders
train_dl, valid_dl, _test_dl = [
    data.DataLoader(
        ds,
        batch_size=args['batch_size'],
        shuffle=(i == 0),
        # drop_last=(i == 0),
        num_workers=8,
        pin_memory=True,
    )
    for i, ds in enumerate(dset_splits)
]


/home/hice1/savunuri3/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [36]:
# Define network
input_dim_1 = next(iter(train_dl))["x_1"].shape[-1]
input_dim_2 = next(iter(train_dl))["x_2"].shape[-1]
model_class = (
    contrastive.ContrastiveEmbedding
)
net = model_class(
    input_dim_1=input_dim_1,
    input_dim_2=input_dim_2,
    shared_dim=args['dim'],
    num_hidden=args['nhidden'],
    lr=args['learning_rate'],
)

/home/hice1/savunuri3/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# # Define logger, write configuration files and data splits
# logger = CSVLogger(save_dir=args['out'], name=args['name'])
# # logger.log_hyperparams(hyperparameters.as_dict())
# write_split_identifiers(
#     train_ids=[dset.pairs[i] for i in split_indices[0]],
#     valid_ids=[dset.pairs[i] for i in split_indices[1]],
#     test_ids=[dset.pairs[i] for i in split_indices[2]],
#     out_file=os.path.join(logger.log_dir, "data_splits.json"),
# )
# net.write_config_json(os.path.join(logger.log_dir, "model_config.json"))
# with open(os.path.join(logger.log_dir, "training_config.json"), "w") as sink:
#     json.dump(vars(args), sink, indent=4)


In [60]:
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
early_stop_callback = EarlyStopping(
    monitor='val_loss',         # metric name to monitor
    patience=50,                 # epochs with no improvement after which to stop
    mode='min',                 # 'min' if you want to minimize the metric
    verbose=True                # optional: prints a message when triggered
)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    save_top_k=1,
    mode='min',
    filename='best-checkpoint'
)


In [61]:
# Train
trainer = pl.Trainer(
    max_epochs=1000,
    accelerator="cuda",
    devices=1,
    enable_progress_bar=True,
    # logger=logger,
    # log_every_n_steps=10,
    deterministic=True,
    callbacks=[early_stop_callback, checkpoint_callback],

)
trainer.fit(net, train_dataloaders=train_dl, val_dataloaders=valid_dl)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type | Params | Mode 
----------------------------------------------
0 | project_1    | MLP  | 33.3 K | train
1 | project_2    | MLP  | 1.2 M  | train
  | other params | n/a  | 1      | n/a  
----------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.865     Total estimated model params size (MB)
14        Modules in train mode
0         Modules in eval mode
SLURM auto-requeueing enabled. Setting signal handlers.


Epoch 0: 100%|██████████| 29/29 [00:00<00:00, 61.43it/s, v_num=6, train_loss=0.215, val_loss=1.470]

Metric val_loss improved. New best score: 1.472


Epoch 2: 100%|██████████| 29/29 [00:00<00:00, 40.72it/s, v_num=6, train_loss=0.0965, val_loss=1.440]

Metric val_loss improved by 0.031 >= min_delta = 0.0. New best score: 1.442


...

Epoch 690: 100%|██████████| 29/29 [00:00<00:00, 53.08it/s, v_num=6, train_loss=1.42e-5, val_loss=0.224]

Metric val_loss improved by 0.002 >= min_delta = 0.0. New best score: 0.224


Epoch 740: 100%|██████████| 29/29 [00:00<00:00, 47.54it/s, v_num=6, train_loss=1.41e-5, val_loss=0.287] 

Monitored metric val_loss did not improve in the last 50 records. Best score: 0.224. Signaling Trainer to stop.


Epoch 740: 100%|██████████| 29/29 [00:00<00:00, 47.38it/s, v_num=6, train_loss=1.41e-5, val_loss=0.287]


In [62]:



# Export model as ONNX files
contrastive.model_to_onnx(
    net.project_1,
    "project_1.onnx",
    input_shape=(input_dim_1,),
)
contrastive.model_to_onnx(
    net.project_2,
    "project_2.onnx",
    input_shape=(input_dim_2,),
)


2025-04-20 00:16:05.619536333 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 418099, index: 0, mask: {1, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-20 00:16:05.619552234 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 418100, index: 1, mask: {2, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-20 00:16:05.619569698 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 418101, index: 2, mask: {3, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-20 00:16:05.621094235 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 418119, index: 20, mask: {21, }, error code: 22 error msg: Invalid argument. Specify the num